## active_editors

update cchen.repo_active_editors table for Superset dashboard monthly

In [12]:
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive
import os

In [13]:
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
metrics_month = pd.Period(METRICS_MONTH_TEXT)

In [14]:
metrics_month

Period('2021-07', 'M')

In [ ]:
## add active editors 
hive.run(
"""
insert into cchen.repo_active_editors 
select
    month,
    'All' as project, 
    'All' AS project_family, 
    'All' AS market,
    count(*) as active_editors,
    sum(cast(registration_month = month as int)) as new_active_editors,
    count(*) - sum(cast(registration_month = month as int)) as returning_active_editors
from (
    select
        cast(month as date) as month,
        user_name,
        sum(content_edits) as content_edits,
        max(bot_by_group) as bot_by_group,
        cast(trunc(min(user_registration), "MONTH") as date) as registration_month
    from neilpquinn.editor_month
    where
        month = CONCAT('{month}','-01') and
        user_id != 0
    group by month, user_name
) global_edits
where
    content_edits >= 5 and
    not bot_by_group and
    user_name not regexp "bot\\b"
group by month""".format(
       month = METRICS_MONTH_TEXT
    ))
                              




In [ ]:
## add diversity 
## Established

hive.run(
"""
with gs_editors as (
    select 
        sum(edit_count) as edit_count,
        sum(namespace_zero_edit_count) as namespace_zero_edit_count,
        max(size(user_is_bot_by) > 0) as bot
    from wmf.editors_daily gd
    left join canonical_data.countries cdc
    on gd.country_code = cdc.iso_code
    where
        month = '{month}' and
        economic_region = "Global North" and
        not user_is_anonymous and 
        gd.action_type = 0
    group by user_fingerprint_or_id
)

insert into cchen.repo_active_editors 
select
    CONCAT('{month}','-01 00:00:00.0') AS month,
    "All" as project,
    "All" as project_family,
    "Global North" as market,
    sum(cast(namespace_zero_edit_count >= 5 and not bot as int)) as active_editors,
    0 as new_active_editors,
    0 as returning_active_editors
from gs_editors""".format(
       month = METRICS_MONTH_TEXT
    ))

In [ ]:
## add diversity 
## Emerging

hive.run(
"""
with gs_editors as (
    select 
        sum(edit_count) as edit_count,
        sum(namespace_zero_edit_count) as namespace_zero_edit_count,
        max(size(user_is_bot_by) > 0) as bot
    from wmf.editors_daily gd
    left join canonical_data.countries cdc
    on gd.country_code = cdc.iso_code
    where
        month = '{month}' and
        economic_region = "Global South" and
        not user_is_anonymous and 
        gd.action_type = 0
    group by user_fingerprint_or_id
)

insert into cchen.repo_active_editors 
select
    CONCAT('{month}','-01 00:00:00.0') AS month,
    "All" as project,
    "All" as project_family,
    "Global South" as market,
    sum(cast(namespace_zero_edit_count >= 5 and not bot as int)) as active_editors,
    0 as new_active_editors,
    0 as returning_active_editors
from gs_editors""".format(
       month = METRICS_MONTH_TEXT
    ))

In [ ]:
## add active editors by project family

hive.run("""
insert into cchen.repo_active_editors 
select
    CONCAT('{month}','-01 00:00:00.0') AS month,
    'All' as project, 
    database_group AS project_family,
    'All' AS market,
    count(*) as active_editors,
    sum(cast(registration_month = month as int)) as new_active_editors,
    count(*) - sum(cast(registration_month = month as int)) as returning_active_editors
from (
    select
        database_group,
        cast(month as date) as month,
        user_name,
        sum(content_edits) as content_edits,
        max(bot_by_group) as bot_by_group,
        cast(trunc(min(user_registration), "MONTH") as date) as registration_month
    from neilpquinn.editor_month
    inner join canonical_data.wikis
    on wiki = database_code
    where
        month = CONCAT('{month}','-01') and
        user_id != 0
        and
        database_group in (
        "commons", "incubator", "mediawiki", "meta",  
        "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
        "wikisource", "wikiversity", "wikivoyage", "wiktionary"
    )
    group by database_group, month, user_name
) global_edits
where
    content_edits >= 5 and
    not bot_by_group and
    user_name not regexp "bot\\b"
group by month, database_group""".format(
       month = METRICS_MONTH_TEXT
    ))

In [ ]:
## add active editors by project

hive.run("""
insert into cchen.repo_active_editors 
select
    CONCAT('{month}','-01 00:00:00.0') AS month,
    wiki as project, 
    database_group AS project_family,
    'All' AS market,
    count(*) as active_editors,
    sum(cast(registration_month = month as int)) as new_active_editors,
    count(*) - sum(cast(registration_month = month as int)) as returning_active_editors
from (
    select
        wiki,
        database_group,
        cast(month as date) as month,
        user_name,
        sum(content_edits) as content_edits,
        max(bot_by_group) as bot_by_group,
        cast(trunc(min(user_registration), "MONTH") as date) as registration_month
    from neilpquinn.editor_month
    inner join canonical_data.wikis on wiki = database_code
    where
        month = CONCAT('{month}','-01') and
        user_id != 0
        and
        database_group in (
        "commons", "mediawiki", "meta",  
        "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
        "wikisource", "wikivoyage", "wiktionary"
    )
    group by wiki,database_group, month, user_name
) global_edits
where
    content_edits >= 5 and
    not bot_by_group and
    user_name not regexp "bot\\b"
group by month, wiki,database_group""".format(
       month = METRICS_MONTH_TEXT
    ))

In [ ]:
os.system("hdfs dfs -chmod -R o+r /user/hive/warehouse/cchen.db/repo_active_editors")